DenseNet-Densely Connected Convolutional Networks
===

# 1.DenseNet介绍
CNN史上的一个里程碑事件是ResNet模型的出现，ResNet可以训练出更深的CNN模型，从而实现更高的准确度。

ResNet模型的核心是通过建立前面层与后面层之间的“短路连接”(shortcuts，skip connection)，这有助于训练过程中梯度的反向传播，从而能训练出更深的CNN网络。

DenseNet基本思路与ResNet一致，但是它建立的是前面所有层与后面层的密集连接(dense connection)，它的名称也是由此而来。

DenseNet的另一大特色是通过特征在channel上的连接来实现特征重用(feature reuse)。这些特点让DenseNet在参数和计算成本更少的情形下实现比ResNet更优的性能。

# 2.DenseNet设计理念
DenseNet提出了一个更激进的密集连接机制：即互相连接所有的层，具体来说就是每个层都会接受其前面所有层作为其额外的输入。下图ResNet的网络联结机制
![image](Images/03/01/05_001.jpg)
下图为DenseNet的密集连接机制
![image](Images/03/01/05_002.jpg)
可以看到，ResNet是每个层与前面的某层(一般是2~3层)短路连接在一起，连接方式是通过元素级相加。

而在DenseNet中，每个层都会与前面所有层在channel维度上连接在一起(这里各个层的特征图大小是相同的),并作为下一层的输入。对于一个$L$层的网络，DenseNet共包含$\frac{L(L+1)}{2}$个连接

相比ResNet，这是一种密集连接。而且DenseNet是直接concat来自不同层的特征图，这可以实现特征重用，提升效率，这一特点是DenseNet与ResNet最主要的区别。

如果用公式表示的话，传统的网络在$l$层的输出为
$$x_l=H_l(x_{l-1})$$
而对于ResNet，增加了来自上一层输入的identity函数
$$x_l=H_l(x_{l-1})+x_{l-1}$$
在DenseNet中，会连接前面所有层作为输入
$$x_l=H_l([x_0,x_1,x_2,...,x_{l-1}])$$
其中，上面的$H_l(\dot)$代表是非线性转化函数(non-liear transformation)，它是一个组合操作，其可能包括一系列的Batch Normalization，ReLU，Pooling及Conv操作。

注意这里$l$层与$l-1$层之间可能实际上包含多个卷积层。DenseNet的前向过程如下图所示，可以更直观地理解其密集连接方式
![image](Images/03/01/05_003.jpg)
比如$h_3$的输入不仅包括来自$h_2$的$x_2$，还包括前面两层的$x_1$和$x_2$，它们是在channel维度上连接在一起的。

CNN网络一般要经过Pooling或者stride>1的Conv来降低特征图的大小，而DenseNet的密集连接方式需要特征图大小保持一致。

为了解决这个问题，DenseNet网络中使用DenseBlock+Transition的结构，其中DenseBlock是包含很多层的模块，每个层的特征图大小相同，层与层之间采用密集连接方式。

而Transition模块是连接两个相邻的DenseBlock，并且通过Pooling使特征图大小降低。图4给出了DenseNet的网路结构，它共包含4个DenseBlock，各个DenseBlock之间通过Transition连接在一起。
![image](Images/03/01/05_004.jpg)

# 3.网络结构
![image](Images/03/01/05_005.jpg)